<a href="https://colab.research.google.com/github/tomonari-masada/course2024-stats2/blob/main/08_logistic_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 一般線形モデル (GLM; generalized linear model)
* 今回はロジスティック回帰を採り上げる。

## 一般線形モデルとは
### 線形回帰
* 線形回帰は、以下のように定式化できる。
$$ Y = \beta_0 + \beta_1 X_1 + \beta_2 X_2 + \cdots + \beta_d X_d + \epsilon $$
* $\beta_0, \beta_1, \ldots, \beta_d$が推定すべき係数。
* $\beta = (\beta_0, \beta_1, \ldots, \beta_d)$、$X = (1, X_1, \ldots, X_d)$とおくと、上式は以下のように書ける。
$$ Y = \beta^\top X + \epsilon $$
* 最小二乗法では、誤差項$\epsilon$が正規分布に従うと仮定し、最尤推定で推定する。


### 線形モデルの一般化
* 線形回帰では、正規分布の平均$\mu$が$\beta^\top X$に等しい、と置いている。
  * つまり、$\mu = E[Y | X] = \beta^\top X$と置いている。
* 正規分布以外の分布でも、その分布の平均を$\beta^\top X$の関数で表すことで、正規分布ではモデリングしにくいデータのモデリングに線形モデルを使う可能性が広がる。

### ロジスティック回帰
* ロジスティック回帰では、正規分布ではなく、ベルヌーイ分布を観測データのモデリングに用いる。
* ベルヌーイ分布の平均$\mu$は、outcomeが$1$となる確率である。
* $\beta^\top X$を$[0,1]$の区間の値に変換するために、シグモイド関数$\mu(s) = \frac{1}{1 + e^{- s}}$を使う。
* ということは、各データ点$\textbf{x}_i = (x_{i,1}, \ldots, x_{i,d})$に対応する、outcomeが$1$となる確率を$p_i = \frac{1}{1 + e^{- \beta^\top \textbf{x}_i}}$として・・・
* 0か1かの正解ラベルを$t_i$と書くと、$\sum_i \{ t_i \log p_i + (1 - t_i) \log (1 - p_i) \}$の最大化によって、最尤推定を行うことができる。
  * ロジスティック回帰の場合のシグモイド関数のような関数を、GLMにおける平均関数(mean function)と呼ぶ。
  * mean functionの逆関数を、リンク関数(link function)と呼ぶ。

### ポアソン回帰
* ポアソン回帰では、正規分布ではなく、ポアソン分布を観測データのモデリングに用いる。
* ポアソン分布の平均$\mu$は、ポアソン分布の唯一のパラメータそのものである。このパラメータが取る値の範囲は$(0,\infty)$である。
* $\beta^\top X$を$[0,1]$の区間の値に変換するために、指数関数を使う。
* ということは、各データ点$\textbf{x}_i = (x_{i,1}, \ldots, x_{i,d})$に対応するポアソン分布のパラメータを$\lambda_i = e^{\beta^\top \textbf{x}_i}$として・・・
* 正解の回数を$c_i$と書くと、$\sum_i \{ c_i \log \lambda_i - \lambda_i \}$の最大化によって、最尤推定を行うことができる。
  * ポアソン回帰の場合の平均関数は、指数関数。
  * ということは、リンク関数は、対数関数。

### 一般線形モデルのベイズ化
* 一般線形モデルのベイズ化は、線形回帰と同様、係数と切片に事前分布を導入することで行われる。

## 準備

In [ ]:
!pip install numpyro

* データセットのサイズが大きいときは、GPUの方が高速かもしれない。

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import jax
import jax.numpy as jnp
from jax import random
import numpyro
import numpyro.distributions as dist
from numpyro.infer import NUTS, MCMC

import arviz as az

%config InlineBackend.figure_format = 'retina'
plt.style.use("bmh")

numpyro.set_platform("gpu")

rng_key = random.PRNGKey(0)

## データセット

* UCI機械学習リポジトリにあるAdult Data Set
  * http://archive.ics.uci.edu/ml/datasets/Adult
* 個人が年間5万ドルを稼ぐかどうかを予測する。

In [ ]:
raw_data = pd.read_csv(
    "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data",
    header=None,
    names=[
        "age",
        "workclass",
        "fnlwgt",
        "education-categorical",
        "educ",
        "marital-status",
        "occupation",
        "relationship",
        "race",
        "sex",
        "captial-gain",
        "capital-loss",
        "hours",
        "native-country",
        "income",
    ],
)

In [ ]:
raw_data.head()

* `fnlwgt` (final weight) は後から何らかのプログラムによって計算されて追加されたものらしい。

In [ ]:
raw_data.describe()

In [ ]:
raw_data.info()

* アメリカのデータに限定して分析する。

In [ ]:
data_us = raw_data[raw_data["native-country"] == " United-States"]

In [ ]:
data_us.sample(5)

* インスタンス数を確認。

In [ ]:
data_us.info()

* incomeが50Kドルより大きいか否かという1/0の情報でincome列を置き換える。

In [ ]:
income = 1 * (data_us.income == " >50K")
income.value_counts()

* 特徴量としてはage, educ, hoursだけを取り出す。

In [ ]:
data = data_us[["age", "educ", "hours"]]

* 年齢を10で割っておく。
  * こうしたほうが、convergenceが良くなるらしい。
  * NumPyroによるMCMCの実行速度も速くなるようだ。
* 10で割った年齢の2乗を新たな特徴量として追加する。
  * なぜ？ （ヒント: 係数はマイナスになることを想定している。）

In [ ]:
data["age"] = data["age"] / 10
data["age2"] = data["age"] ** 2
# incomeの列が最後に来るようにするためここで代入している。
data.loc[:,"income"] = income

In [ ]:
data["educ"].value_counts().sort_index()

In [ ]:
data.describe()

## EDA

In [ ]:
sns.pairplot(data);

In [ ]:
# Compute the correlation matrix
corr = data.corr()

# Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=np.bool_)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(7, 6))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(
    corr,
    mask=mask,
    cmap=cmap,
    vmax=1.0,
    linewidths=0.5,
    cbar_kws={"shrink": 0.5},
    annot=True,
    ax=ax,
);

## モデル
* incomeを目的変数とする。1/0の情報をベルヌーイ分布でモデリング。
* そのベルヌーイ分布のパラメータである1が出る確率を、線形モデルでモデル化。
* 確率なので、リンク関数はlogit関数。
  * logit関数は、ロジスティック関数の逆関数。
  * 線形モデルの出力を、ロジスティック関数に与えて、確率に変換。
* 線形モデルでは、各個人のage, educ, hours特徴量を使用。
  * ただし、上述のように、ageの2乗も説明変数として追加されている。

$$ z_i = \beta_0 + \beta_1 X_{age} + \beta_2 X_{age^2} + \beta_3 X_{educ} + \beta_4 X_{hours} $$
$$ y_i \sim \text{Bernoulli}(p_i) \ \ \mbox{where} \ p_i = \frac{1}{1 + e^{-z_i}}$$

* NumPyroでロジスティック回帰を書く方法
 * https://num.pyro.ai/en/stable/handlers.html

In [ ]:
def model(data, labels=None, num_features=1):
  coefs = numpyro.sample('coefs', dist.Normal(jnp.zeros(num_features), jnp.ones(num_features)))
  intercept = numpyro.sample('intercept', dist.Normal(0., 10.))
  logits = jnp.sum(coefs * data, axis=-1) + intercept
  return numpyro.sample('obs', dist.Bernoulli(logits=logits), obs=labels)

## MCMC

In [ ]:
features = ["age", "educ", "hours", "age2"]
X = data[features].values.astype(float)
y = data.income.values.astype(int)

In [ ]:
rng_key, rng_key_ = random.split(rng_key)
kernel = NUTS(model)
mcmc = MCMC(kernel, num_warmup=1000, num_samples=2000, num_chains=2)
mcmc.run(rng_key_, data=X, labels=y, num_features=len(features))

## サンプルの可視化

In [ ]:
mcmc.print_summary()

In [ ]:
fitted = az.from_numpyro(mcmc)

In [ ]:
az.plot_trace(fitted, compact=False);

In [ ]:
az.summary(fitted)

> One of the major benefits that makes Bayesian data analysis worth the extra computational effort in many circumstances is that we can be explicit about our uncertainty. Maximum likelihood returns a number, but how certain can we be that we found the right number? Instead, Bayesian inference returns a distribution over parameter values.

* https://docs.pymc.io/en/v3/pymc-examples/examples/generalized_linear_models/GLM-logistic.html#Some-results
  * このURLは、現在は無効となっている。

* featureごとにサンプルを分ける。

In [ ]:
trace = {}
for feature in features:
  trace[feature] = np.array(
      mcmc.get_samples()["coefs"][:,features.index(feature)]
  )
trace["intercept"] = np.array(mcmc.get_samples()["intercept"])

* 年齢の係数のサンプルと教育年数の係数のサンプルとで、joint plotを描いてみる。

In [ ]:
plt.figure(figsize=(9, 7))
sns.jointplot(x=trace["age"], y=trace["educ"], kind="hex", color="#4CB391")
plt.xlabel("beta_age")
plt.ylabel("beta_educ");

* ロジットの差はオッズ比の対数に当たる。
$$\log\Big(\frac{p}{1-p}\Big) - \log\Big(\frac{p^\prime}{1-p^\prime}\Big) = \log \frac{ p / (1-p) }{ p^\prime / (1 - p^\prime) }$$
* よって、GLMでリンク関数がlogit関数のとき、**線形モデルの係数のexponentialは、オッズ比**の意味を持つ。
  * その係数に対応する説明変数の値が1増えると、線形モデルの出力が係数分だけ増える。
  * つまり、係数の値のexponentialは、その係数に対応する説明変数の値が1増えたときの、増やす前に対する、オッズ比。

In [ ]:
plt.hist(np.exp(trace["educ"]), bins=20, density=True)
plt.xlabel("Odds Ratio");

In [ ]:
lb, ub = np.percentile(trace["educ"], 2.5), np.percentile(trace["educ"], 97.5)
print(f"P({np.exp(lb):.3f} < O.R. < {np.exp(ub):.3f}) = 0.95")

## 予測のためのスパゲッティ・プロット

* incomeが50Kドルを超える確率を、横軸を年齢にして、プロットする。
  * 教育年数が異なると、確率がどのように異なるかを見る。
  * 労働時間は50時間で固定する。


In [ ]:
age = np.linspace(20, 80, 61) / 10.0
new_data = pd.DataFrame({
    "age": np.tile(age, 3),
    "educ": np.repeat([12, 16, 19], len(age)),
    "hours": 50.0,
    "age2": np.tile(age, 3) ** 2,
})

In [ ]:
new_data

In [ ]:
posterior_logit = jnp.expand_dims(trace["intercept"], 0)
for column in data.columns:
  if column == "income": continue
  posterior_logit += (
      jnp.expand_dims(new_data[column].values, -1)
      * jnp.expand_dims(trace[column], 0)
  )

In [ ]:
posterior_logit = posterior_logit[:, ::20]

In [ ]:
posterior_logit.shape

* 教育年数で色分けしてプロットする。

In [ ]:
def my_plot(ax, xs, ys, *args, **kwargs):
  ax.plot(xs,ys, *args, **kwargs)
  if "label" in kwargs.keys():
    handles, labels = plt.gca().get_legend_handles_labels()
    newLabels, newHandles = [], []
    for handle, label in zip(handles, labels):
      if label not in newLabels: # remove duplicates
        newLabels.append(label)
        newHandles.append(handle)
        handle.set_alpha(1) # set alpha=1 for legend
    plt.legend(newHandles, newLabels)


_, ax = plt.subplots(figsize=(7, 5))

for i, educ in enumerate([12, 16, 19]):
    idx = new_data.index[new_data["educ"] == educ].tolist()
    my_plot(
        ax,
        age * 10, 1 / (1 + np.exp(- posterior_logit[idx,:])),
        alpha=0.04,
        label=f"educ={i}",
        color=f"C{i}",
    )

ax.set_ylabel("P(income > 50K | age)")
ax.set_xlabel("Age")
ax.set_ylim(0, 1)
ax.set_xlim(20, 80);